# LulaSafe API

This tutorial will guide you step by step on how to use the API having the client code generated from OpenAPI specification.

In [ ]:
import os
import json
import urllib
import json
import sys
import time
from datetime import datetime

def run_command(cmd):
    print(os.popen(cmd).read())

api_client_path = "python/.api-lulasafe/"

## Client code generation

Install NPM package

In [ ]:
run_command('npm install -g @openapitools/openapi-generator-cli')

and generate the client:

In [ ]:
current_directory = os.getcwd()
path_to_yaml_schema = os.path.join(
    current_directory, '..', 'openapi', 'lulasafe.yaml')
run_command('npx openapi-generator-cli version-manager set 6.1.0')
run_command(
    f'npx openapi-generator-cli generate -g python -i "{path_to_yaml_schema}" -o "{api_client_path}"')

## Package restore
```
cd python/.api-lulasafe
pip3 install -r requirements.txt
pip3 install .
```

In [ ]:
os.chdir(api_client_path)
run_command("pip3 install -r requirements.txt")
run_command("pip3 install .")
os.chdir('../../')

> **Note**
>
> In `python/.api-lulasafe` you will see a [README.md](python/.api-lulasafe/README.md) with API documentation. Including all types

## Authentication
> **Warning**
>
> Until we add support for OpenID Connect client credentials flow, we need to perform some custom token retrieving actions

### 1. Read your credentials
> **Important**
>
> Create [`appsettings.json`](../appsettings.json) in the repo root and set your credentials into it
>
> ``` JSON
> {
>     "ClientId": "< Your Lula login >",
>     "ClientSecret": "< Your Lula password >"
> }
> 

In [ ]:
class LulaSafeOptions():
    def __init__(self, client_id, client_secret):
        self.client_id = client_id
        self.client_secret = client_secret

    @staticmethod
    def read_from_config(filepath):
        with open(filepath) as app_settings:
            settings_object = json.load(app_settings)
            return LulaSafeOptions(settings_object["ClientId"], settings_object["ClientSecret"])

    @staticmethod
    def read_from_default_config():
        return LulaSafeOptions.read_from_config("../appsettings.json")

In [ ]:
# Lula URLs
lula_url = "https://api.staging-lula.is/"
init_session_path = "v1/login/initialize"
submit_login_path = "v1/login/submit"

### 2. Initiate session

In [ ]:
request = urllib.request.Request(url=f"{lula_url}{init_session_path}")
response = urllib.request.urlopen(request).read()
json_response = json.loads(response)

flow_id = json_response["id"]
f"FlowId '{flow_id}'"

### 3. Get session token used as bearer

In [ ]:
lula_safe_config = LulaSafeOptions.read_from_default_config()

request_data = {
    "method": "password",
    "password_identifier": lula_safe_config.client_id,
    "password": lula_safe_config.client_secret
}
full_url = f"{lula_url}{submit_login_path}?flow={flow_id}"

json_request = json.dumps(request_data).encode('utf-8')
request = urllib.request.Request(
    data=json_request, method="POST", url=full_url)
request.add_header('Content-Type', 'application/json; charset=utf-8')

response = urllib.request.urlopen(request).read()
json_response = json.loads(response)
bearer_token = json_response["session_token"]
f"Bearer Token '{bearer_token}'"

## Client usage
We will generate a client from OpenAPI and use it.


In [ ]:
from openapi_client.model.stripe_identity_verification_credentials_response import StripeIdentityVerificationCredentialsResponse
from openapi_client.model.issuer_state import IssuerState
from openapi_client.model.email import Email
from openapi_client.model.get_driver_assessment_results_response import GetDriverAssessmentResultsResponse
from openapi_client.model.validation_problem_details import ValidationProblemDetails
from openapi_client.model.problem_details import ProblemDetails
from openapi_client.model.driver_assessment_request import DriverAssessmentRequest
from openapi_client.model.driving_license_id import DrivingLicenseId
from openapi_client.model.driving_license import DrivingLicense
from openapi_client.model.assessee import Assessee
from openapi_client.model.address import Address
from openapi_client.api import default_api as lula_safe_api
import openapi_client
os.sys.path.append(api_client_path)

configuration = openapi_client.Configuration(host=f"{lula_url}risk/v0.1-beta1")

There are two ways to generate an OpenAPI client in python:  
1. Classic one, where you take responsibility to close the client connection  
2. Modern one, that closes it automatically  

#### Classic option

In [ ]:
# classic_client = openapi_client.ApiClient(configuration)
# ....
# classic_client.close()

#### Modern option

In [ ]:
# with openapi_client.ApiClient(configuration) as client:
# ....
#    lula_safe_client = lula_safe_api.DefaultApi(client)

# openapi_client instance called `client` will be closed automatically when
# code reaches out of the request scope

## Session concept
As long as API must also be usable from client side application (i.e. from browser) first you establish a short leaved session from a back-end. Then you can pass it to front-end and do not worry about it's disclosure. Or use it from back-end too.
So after you got a session Id, use it for later calls.

### Establishing a session
Use bearer token as usual in `Authorization` header to establish a session

In [ ]:
session_id = None

with openapi_client.ApiClient(configuration, header_name="Authorization", header_value=f"Bearer {bearer_token}") as client:
    # Create a new instance
    lula_safe_client = lula_safe_api.DefaultApi(client)

    try:
        session_response = lula_safe_client.create_session()
        session_id = session_response["session_id"]
    except openapi_client.ApiException as e:
        print("Exception occurred while calling LulaApi->create_session:%s\n" % e)
f"SessionId '{session_id}'"

> **Note**
>
> Having a `sessionId` you no longer need to pass a bearer token. Hence you can pass it to front-end or mobile app

### Driver Assessment
> **Important**
>
>  Store assessment Id on your back-end to later retrieve the result again

Collect driver data and request an assessment for that driver

In [ ]:
assessee_data = Assessee(
    first_name="Antonio",
    last_name="Bernette",
    middle_name="",
    date_of_birth=datetime(1982, 11, 17).date(),
    phone="270-555-7152",
    email=Email("antonio@email.com")
)
driving_license_id = DrivingLicenseId("111119615")
driving_license = DrivingLicense(
    id=driving_license_id,
    expiry_date=datetime(2024, 10, 20).date(),
    issuer_state=IssuerState("KY")
)
address = Address(
    line1="7104 Cadillac Boulevard",
    line2="",
    city="Arlington",
    state="TX",
    country="US",
    zip_code="76016"
)
driver_assessment_body = lula_safe_api.DriverAssessmentRequest(
    assessee=assessee_data,
    driving_license=driving_license,
    address=address
)


### Handle non-success status codes

Catch generic `ApiException` and check status code

In [ ]:
def handle_lula_error(e: openapi_client.ApiException):
    if (e.status == 400):
        problem_details: ProblemDetails = e.body
        print(problem_details)
    elif (e.status == 404):  # SessionNotFound, no body
        problem_details: ProblemDetails = e.body
        print(problem_details)
    elif (e.status == 410):  # SessionExpired
        problem_details: ProblemDetails = e.body
        print(problem_details)
    elif (e.status == 422):  # Incorrect Parameters Supplied
        response_body = e.body
        response_object = json.loads(response_body)
        errors = response_object["errors"]
        for key, error_values in errors.items():
            print(f"'{key}':")
            for error in error_values:
                print(f"\t{error}")
    else:
        print(f"Unknown error:{e.status} code with body:\n{e.body}")

In [ ]:
driver_assessment_id = None

with openapi_client.ApiClient(configuration) as client:
    # Another way to provide Bearer token
    client.set_default_header("Authorization", f"Bearer {bearer_token}")
    lula_safe_client = lula_safe_api.DefaultApi(client)
    try:
        driver_assessment: GetDriverAssessmentResultsResponse = lula_safe_client.request_driver_assessment(
            session_id=session_id,
            driver_assessment_request=driver_assessment_body
        )
        driver_assessment_id = driver_assessment["assessment"]["value"]["id"]
    except openapi_client.ApiException as e:
        handle_lula_error(e)

driver_assessment

### Document and selfie verification

To use document and selfie on a front-end you need it's credentials. Here they are

In [ ]:
stripe_identity_publishable_key = None
with openapi_client.ApiClient(configuration) as client:
    client.set_default_header("Authorization", f"Bearer {bearer_token}")
    lula_safe_client = lula_safe_api.DefaultApi(client)

    try:
        stripe_response: StripeIdentityVerificationCredentialsResponse = lula_safe_client.get_stripe_identity_verification_credentials(
            session_id=session_id,
            driver_assessment_id=driver_assessment_id
        )
        stripe_identity_publishable_key = stripe_response["stripe_identity_publishable_key"]
    except openapi_client.ApiException as e:
        handle_lula_error(e)

f"StripeIdentityPublishableKey '{stripe_identity_publishable_key}'"

### Getting assessment results later
Get any previous assessment results by assessment Id

In [ ]:
risk_conclusion = None
criminal_status_check = None
document_status_check = None
identity_status_check = None
mvr_status_check = None


with openapi_client.ApiClient(configuration) as client:
    client.set_default_header("Authorization", f"Bearer {bearer_token}")
    lula_safe_client = lula_safe_api.DefaultApi(client)

    try:
        # Because generator propagates type as not nullable, request will fail without `_check_return_type = False` kwarg
        assessment_result = lula_safe_client.get_driver_assessment_by_id(
            driver_assessment_id=driver_assessment_id,
            _check_return_type=False
        )
        criminal_status_check = assessment_result["criminal_check"]["status"]
        document_status_check = assessment_result["document_check"]["status"]
        identity_status_check = assessment_result["identity_check"]["status"]
        mvr_status_check = assessment_result["mvr_check"]["status"]

        risk_conclusion = assessment_result["lula_safe_conclusion"]["risk"]
    except openapi_client.ApiException as e:
        handle_lula_error(e)

f"""Criminal check status '{criminal_status_check}';
Document check status '{document_status_check}';
Identity check status '{identity_status_check}';
MVR check status '{mvr_status_check}';
LulaSafe Conclusion '{risk_conclusion}'.
"""